In [2]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from scipy.interpolate import griddata
from scipy.interpolate import NearestNDInterpolator
import os
import tempfile
from typing import List
import xtgeo
import numpy as np
from pymongo import MongoClient
from bson import ObjectId
import logging
import pandas as pd
from vtk import vtkCellArray, vtkExplicitStructuredGrid, vtkPoints, reference, vtkCellCenters, vtkPolyData, vtkDoubleArray, vtkIdList, vtkGenericCell, vtkCellLocator
from vtkmodules.util.numpy_support import numpy_to_vtkIdTypeArray, numpy_to_vtk,vtk_to_numpy


In [3]:
def seismic_ascii_parser(file_path):
    all_data = []
    head = ""
    xarray = []
    yarray = []
    zarray = []
    with open(file_path, 'r') as f:
        content = f.readlines()
        for line in content:
            if '#' in line:
                head += line
                continue
            l = line.split(' ')
            xarray.append(float(l[0]))
            yarray.append(float(l[1]))
            zarray.append(float(l[2]))

    df = pd.DataFrame({'x': xarray, 'y': yarray, 'z': zarray})
    all_data.append({"test": df})
    return create_seismic_ascii_date(all_data)[0]

def create_seismic_ascii_date(data_frame):
    for dataframe_data in data_frame:
        _, value = next(iter(dataframe_data.items()))
        return value.to_dict("records"),


In [4]:
def cube_grid_parser(data_folder: str,
                     input_files: List[str]):
    """
    Преобразует файл с содержанием COORD, ZCORN, ACTNUM (сетка куба) в словарь.
    :param data_folder: Путь к папке с данными.
    :param input_files: Список файлов, поддерживаемых форматом.
    :param parameters: Список параметров в виде {имя параметра: значение}, необходимых ридеру для чтения файлов.
    :param primary_data: Список мнемоник.
    """
    check_mas = ["COORD", "ZCORN", "ACTNUM", "SPECGRID", "COORDSYS"]
    errors = {"read_grid": ""}
    errors_counter = 1
    if len(input_files) > 1:
        with tempfile.NamedTemporaryFile(suffix=".grdecl", delete=False) as temp_file:
            full_path = temp_file.name
            with open(full_path, 'w') as f_out:
                for file in input_files:
                    with open(os.path.join(data_folder, file), 'r') as f:
                        data = remove_comment_lines(f.read()).replace("NOECHO","").replace("ECHO","")
                        f_out.write(data)
                        f_out.write("\n")
    else:
        full_path = os.path.join(data_folder, input_files[0])

    _, keys = find_key(full_path)
    for key in check_mas:
        if key not in keys:
            errors["read_grid"] += f"{errors_counter}) Отсутвует ключ {key}, дозагрузите нужный файл\n"
            errors_counter+=1

    if errors_counter > 1:
        return {}, errors

    grid = xtgeo.grid_from_file(full_path, fformat="grdecl")

    res = np.all((grid.actnum_array >= 0) & (grid.actnum_array <= 1))
    if not res:
        errors["read_grid"] += f"{errors_counter}) Значение ACTNUM не входит в диапазон от 0 до 1"
        errors_counter += 1
        return {}, errors
    i_max = grid.ncol
    j_max = grid.nrow
    k_max = grid.nlay
    #grid_idx = (np.arange(i_max * j_max * k_max)
    #            .reshape(k_max, j_max, i_max, order='F').transpose(-1, 1, 0))

    _, vert_arr, conn_arr, _ = grid.get_vtk_esg_geometry_data()
    #vert_arr = np.asfortranarray(vert_arr)
    conn_arr = conn_arr.astype('int64')
    vol_active_cells = grid.get_bulk_volume().get_npvalues1d(activeonly = True ,  order = 'F')

    #оцениваем качество сетки
    grid.get_gridquality_properties()
    df_grid = grid.get_dataframe(activeonly=True)
    df_grid[['IX', 'JY', 'KZ']] = df_grid[['IX', 'JY', 'KZ']].astype(int)

    #1. треугольные ячейки
    idx_cells = df_grid[(df_grid['minangle_topbase']<1)|(df_grid['maxangle_topbase']>179)
                           |(df_grid['minangle_topbase_proj']<1)|(df_grid['maxangle_topbase_proj']>179)
                           |(df_grid['minangle_sides']<1)|(df_grid['maxangle_sides']>179)].index.tolist()
    ijk_er = df_grid.loc[idx_cells][['IX', 'JY', 'KZ']].values
    idx_err_triang = ijk_er[:,2]*i_max*j_max+ijk_er[:,1]*i_max +ijk_er[:,0]

    #2. Вогнутые ячейки, если смотреть сверху.
    # Ячейка является вогнутой, если один угол находится внутри треугольника, образованного другими углами сверху и/или основания.
    # Проверяются только координаты X, Y
    idx_cells = df_grid[(df_grid['concave_proj']>0)].index.tolist()
    ijk_er = df_grid.loc[idx_cells][['IX', 'JY', 'KZ']].values
    idx_err_concave_proj = ijk_er[:,2]*i_max*j_max+ijk_er[:,1]*i_max +ijk_er[:,0]

    #3. схлопнутые по оси z ячейки
    idx_cells = df_grid[df_grid['collapsed']> 0].index.tolist()
    ijk_er = df_grid.loc[idx_cells][['IX', 'JY', 'KZ']].values
    idx_err_collapsed = ijk_er[:,2]*i_max*j_max+ijk_er[:,1]*i_max +ijk_er[:,0]

    #4. отрицательная толщина (по оси z считается)
    idx_cells = df_grid[(df_grid['negative_thickness']>0)].index.tolist()
    ijk_er = df_grid.loc[idx_cells][['IX', 'JY', 'KZ']].values
    idx_err_negative_thickness = ijk_er[:,2]*i_max*j_max+ijk_er[:,1]*i_max +ijk_er[:,0]

    return {
        "spec_grid": [i_max, j_max, k_max],

        'active_cells': grid.get_actnum_indices(order='F', inverse=False).tolist(),

        'xyz_vertices': vert_arr.tolist(),
        'connectivity_array_cell': conn_arr.tolist(),
        'volumes_active_cells': vol_active_cells.tolist(),
        'triangle_cells': idx_err_triang.tolist(),
        'concave_proj_cells': idx_err_concave_proj.tolist(),
        'collapsed_cells': idx_err_collapsed.tolist(),
        'negative_thickness_cells': idx_err_negative_thickness.tolist()
    }, errors


In [5]:
def extract_unique_coordinates(coord_list):
    """
    Функция находит все неповторяющеся координаты по инлайнам и крослайнам
    """
    x_set = set()
    y_set = set()

    for coord in coord_list:
        x_set.add(coord['x'])
        y_set.add(coord['y'])

    x_unique = list(x_set)
    y_unique = list(y_set)

    return x_unique, y_unique



In [6]:
def remove_comment_lines(data, commenter='--') -> str:
    data_lines = data.strip().split('\n')
    newdata = []
    for line in data_lines:
        if not line.strip():
            continue
        elif line.find(commenter) != -1:
            newline = line[0:line.find(commenter)].strip()
            if len(newline) == 0:
                continue
            newdata.append(newline)
        else:
            newdata.append(line)
    return '\n'.join(newdata)

def find_key(file_path: str) -> [bool, str]:
    with open(file_path, 'r') as f:
        contents = f.read()
        contents = remove_comment_lines(contents, commenter='--')
        contents_in_block = contents.strip().split('/')
        keys_contents_in_block = [x.split()[0] for x in contents_in_block if x]
        NumKeywords = len(contents_in_block)
        if NumKeywords == 0:
            return False, ""
        else:
            return True, keys_contents_in_block


In [7]:
def show_graf(data1, data2, df):
    """
    data1 - структурная карта кровли
    data2 - структурная карта подошвы
    df - датафрейм, полученный из грида, содержащий координаты [Х, У, нижняя область грида, верхняя область грида]
    """
        
    # Создание фигуры
    fig = go.Figure()

    x1 = [point['x'] for point in data1]
    y1 = [point['y'] for point in data1]
    z1 = [point['z'] for point in data1]
        
    x2 = [point['x'] for point in data2]
    y2 = [point['y'] for point in data2]
    z2 = [point['z'] for point in data2]
     
    fig.add_trace(go.Scatter3d(
        x=x1,
        y=y1,
        z=z1,
        mode='markers',
        marker=dict(size=3, color='blue'),
        name='Top_TWT'
    ))
        
    fig.add_trace(go.Scatter3d(
        x=x2,
        y=y2,
        z=z2,
        mode='markers',
        marker=dict(size=3, color='red'),
        name='Bottom_TWT'
    ))
    


    fig.add_trace(go.Scatter3d(
        x=df['x'],
        y=df['y'],
        z=df['z_min'],
        mode='markers',
        name='Нижняя граница Грида',
        marker=dict(color='yellow', size=2)
    ))

    fig.add_trace(go.Scatter3d(
        x=df['x'],
        y=df['y'],
        z=df['z_max'],
        mode='markers',
        name='Верхняя граница Грида',
        marker=dict(color='yellow', size=2)
    ))
    

    fig.update_layout(
        title='Визуализация структурных карт и верхней и нижней областей грида',
        width=800,  
        height=600,  
        scene=dict(
            xaxis_title='X',
            yaxis_title='Y',
            zaxis_title='Z'
        )
    )
    
    # Показать график
    fig.show()


In [8]:
def procent_vihoda (df1, df2, oblast):
    """
    функция считает процент от количества точек df1, которые находятся выше области точек df2
    :param df1: датафрейм структурной карты
    :param df2: датафрейм верхней или нижней области грида
    oblast: имя области грида, которе мы сравниваем "z_max" или "z_min"
    :return: процент. 
    В случае нижней границы грида и карты подошвы полученный процент вычитается из 100
    """
   
   
    # Интерполяция значений z из первой области на координаты второй области
    z_interpolated = griddata(
        (df1['x'], df1['y']), 
        df1['z'], 
        (df2['x'], df2['y']), 
        method='linear'
    )
    
    # Сравнение значений z
    # Убираем NaN значения, если интерполяция не смогла найти соответствие
    valid_indices = ~np.isnan(z_interpolated)
    above_count = (z_interpolated[valid_indices] > df2[oblast][valid_indices]).sum()
    total_count = valid_indices.sum()
    
    # Вычисление процента точек первой области выше второй
    if total_count > 0:
        percentage_above = (above_count / total_count) * 100
    else:
        percentage_above = 0
    
    return round(percentage_above, 2) 
    



In [9]:
def calculate_percentage(df1, df2, flag):
    # Создание интерполятора
    interpolator = NearestNDInterpolator((df1['x'], df1['y']), df1['z'])
    
    # Интерполяция значений z из df1 на координаты из df2
    z_interpolated = interpolator(df2['x'], df2['y'])

    # Создание ячеек между z_min и z_max
    z_min = df2['z_min'].values
    z_max = df2['z_max'].values
    cell_height = (z_max - z_min) / res[0]["spec_grid"][2]  # Высота ячейки

    # Подсчет количества ячеек
    count_above = 0
    total_cells = 0

    for i in range(len(z_min)):
        for j in range(res[0]["spec_grid"][2]+1):  # 48 ячеек
            cell_top = z_min[i] + (j + 1) * cell_height[i]
            cell_bottom = z_min[i] + j * cell_height[i]
            if flag == 0:  # Процент ячеек выше области df1
                if z_interpolated[i] < cell_bottom:
                    count_above += 1
            elif flag == 1:  # Процент ячеек ниже области df1
                if z_interpolated[i] > cell_top:
                    count_above += 1
            total_cells += 1

    # Вычисление процента
    if total_cells > 0:
        percentage = (count_above / total_cells) * 100
    else:
        percentage = 0.0

    return round(percentage, 2)


In [12]:
TWT_Bottom = seismic_ascii_parser(r"C:/HV/Seismic/datas/TWT_Bottom_U1.txt")
TWT_Top = seismic_ascii_parser(r"C:/HV/Seismic/datas/TWT_Top_U1.txt")
res = cube_grid_parser(r"C:/HV/Seismic/datas/grid", ["GRID.GRDECL", "GRID_ACTNUM.GRDECL", "GRID_COORD.GRDECL", "GRID_ZCORN.GRDECL"])
connectivity_array_cell = np.array(res[0]["connectivity_array_cell"], dtype=np.int64)  
xyz_vertices = np.array(res[0]["xyz_vertices"])
spec_grid = np.array(res[0]["spec_grid"])

In [13]:
#1.Вычисляем центры ячеек по оси z и строим каркасную сетку vtk
z_center_cells = np.sum(xyz_vertices[connectivity_array_cell.reshape(-1,8),2], axis=1)/8
vert_arr = xyz_vertices
vtk_points = vtkPoints()
vtk_points.SetData(numpy_to_vtk(vert_arr, deep=1))
vtk_cell_array = vtkCellArray()
vtk_cell_array.SetData(8, numpy_to_vtkIdTypeArray(connectivity_array_cell, deep=1))

vtk_esgrid = vtkExplicitStructuredGrid()
vtk_esgrid.SetDimensions(spec_grid+1)
vtk_esgrid.SetPoints(vtk_points)
vtk_esgrid.SetCells(vtk_cell_array)


In [27]:
"""
Создаем свой грид в виде двух областей [x, y, z_min, z_max] размерности 186х214 ячеек над кровлей и подошвой
"""
x_coords, y_coords = extract_unique_coordinates(TWT_Top)
start_x = min(x_coords)
stop_x = max(x_coords)
start_y = min(y_coords)
stop_y = max(y_coords)
delta_x = (max(x_coords) - min(x_coords)) / 186
delta_y = (max(y_coords) - min(y_coords)) / 214

grid = []
for i in range(186):
    #grid.append([])
    for j in range(214):
        grid.append([start_x + i*delta_x, start_y + j*delta_y, -2350, -2260])

df = pd.DataFrame(np.array(grid), columns=['x', 'y', 'z_min', 'z_max'])

In [ ]:
show_graf(TWT_Top, TWT_Bottom, df)

In [ ]:
pr_top = procent_vihoda(pd.DataFrame(TWT_Top), df[['x', 'y', 'z_max']], 'z_max')
pr_bot = procent_vihoda(pd.DataFrame(TWT_Bottom), df[['x', 'y', 'z_min']], 'z_min')

if pr_top == 100:
    print('Ни одна точка структурного каркаса не попадает в область грида. тест не пройден, т.к. все точки сруктурного каркаса кровли выше области Грида')
if pr_top == 0:
    print('Ни одна точка структурного каркаса не попадает в область грида. тест не пройден, т.к. все точки сруктурного каркаса подошвы ниже области Грида')

pr_top_1 = calculate_percentage(pd.DataFrame(TWT_Top), df, 0)
pr_bot_1 = calculate_percentage(pd.DataFrame(TWT_Bottom), df, 1)
print(pr_top_1 + pr_bot_1, '% ячеек грида, выходят за пределы структурного каркаса')
print('Пространство структурного каркаса и грида пересекаются на ', 100 - pr_top_1 - pr_bot_1, '% от объёма грида')
  


In [ ]:
"""
Из-за слишком большого значеня точек у меня не досчитывается Грид. Мне нужно получить из него датафрейм с координатами и верхней и нижней границей
"""

# Извлекаем координаты и создаем датафрейм
data = []
for i in range(connectivity_array_cell.shape[0]):
    cell_indices = connectivity_array_cell[i]
    cell_vertices = xyz_vertices[cell_indices]
    
    z_min = np.min(cell_vertices[2])  # Минимальное значение z в ячейке
    z_max = np.max(cell_vertices[2])  # Максимальное значение z в ячейке
    x_center = np.mean(cell_vertices[0])  # Центр по x
    y_center = np.mean(cell_vertices[1])  # Центр по y
    
    data.append({'x': x_center, 'y': y_center, 'z_min': z_min, 'z_max': z_max})
df = pd.DataFrame(data)